# Dataset Construction

In [1]:
import numpy as np
import pandas as pd
from torch_geometric_temporal.signal import StaticGraphTemporalSignal
import torch

class StaticSP500DatasetLoader(object):
    def __init__(self, correlation_type='pearsons'):
        self._read_csv(correlation_type)

    def _read_csv(self, correlation_type):
        self._correlation_matrix = np.fromfile(f's&p500_{correlation_type}.csv', sep=',')
        N = int(np.sqrt(len(self._correlation_matrix)))
        self._correlation_matrix = self._correlation_matrix.reshape(N, N)
        _correlation_threshold = 0.9
        self._correlation_matrix[self._correlation_matrix < _correlation_threshold] = 0
        
        df = pd.read_csv('s&p500.csv')
        df = df.set_index('Date')
        data = torch.from_numpy(df.to_numpy()).to(torch.float32)
        ratios = torch.zeros_like(data)
        # calculate daily return ratio
        for d in range(1, data.size(0)):
            ratios[d-1] = (data[d] - data[d-1]) / data[d-1]
        # skip the first day which cannot calculate daily return ratio
        # and round data size to nearest multiple of batch_size
        days_in_quarter = 64
        num_quarters = data.size(0) // days_in_quarter
        num_days = num_quarters * days_in_quarter
        ratios = ratios[:num_days]
        self._dataset = ratios.numpy()

    def _get_edges(self):
        self._edges = np.array(self._correlation_matrix.nonzero())

    def _get_edge_weights(self):
        self._edge_weights = self._correlation_matrix[self._correlation_matrix > 0]

    def _get_targets_and_features(self):
        stacked_target = self._dataset
        self.features = [
            stacked_target[i : i + self.lags, :].T
            for i in range(stacked_target.shape[0] - self.lags)
        ]
        self.targets = [
            stacked_target[i + self.lags, :].T
            for i in range(stacked_target.shape[0] - self.lags)
        ]

    def get_dataset(self, lags: int) -> StaticGraphTemporalSignal:
        """Returning the data iterator.

        Args types:
            * **lags** *(int)* - The number of time lags.
        Return types:
            * **dataset** *(StaticGraphTemporalSignal)*
        """
        self.lags = lags
        self._get_edges()
        self._get_edge_weights()
        self._get_targets_and_features()
        dataset = StaticGraphTemporalSignal(
            self._edges, self._edge_weights, self.features, self.targets
        )
        return dataset

In [2]:
from torch_geometric_temporal.signal import temporal_signal_split

device = 'cpu'

loader = StaticSP500DatasetLoader()

lags = 63

dataset = loader.get_dataset(lags)

train_dataset, test_val_dataset = temporal_signal_split(dataset, train_ratio=0.8)
val_dataset, test_dataset = temporal_signal_split(test_val_dataset, train_ratio=0.5)

# Evaluation

In [3]:
import math
from torcheval.metrics.functional import binary_f1_score

def mse(y_hats, ys):
    return torch.nn.functional.mse_loss(y_hats, ys)

def rmse(y_hats, ys):
    return math.sqrt(mse(y_hats, ys))

def mae(y_hats, ys):
    return torch.nn.functional.l1_loss(y_hats, ys)

def accuracy(y_hats, ys):
    return ((y_hats > 0) == (ys > 0)).sum().item() / y_hats.numel()

def f1(y_hats, ys):
    return binary_f1_score((y_hats > 0).int().flatten(), (ys > 0).int().flatten(), threshold=1).item()


# RGCN

In [4]:
import torch
import torch.nn.functional as F
from torch_geometric_temporal.nn.recurrent import DCRNN

class RecurrentGCN(torch.nn.Module):
    def __init__(self, node_features):
        super(RecurrentGCN, self).__init__()
        self.recurrent = DCRNN(node_features, 32, 1)
        self.linear = torch.nn.Linear(32, 1)

    def forward(self, x, edge_index, edge_weight):
        h = self.recurrent(x, edge_index, edge_weight)
        h = F.relu(h)
        h = self.linear(h)
        return h

In [5]:
from tqdm import tqdm
import wandb

model = RecurrentGCN(node_features = lags).to(device)

lr = 0.005
num_epochs = 50

optimizer = torch.optim.Adam(model.parameters(), lr=lr)

track_with_wandb = True

if track_with_wandb:
    wandb.init(project="cs224w-stock-market-prediction", config={
        "dataset": "S&P500",
        "learning_rate": lr,
        "epochs": num_epochs,
        "architecture": "DCRNN",
    })

for epoch in tqdm(range(num_epochs)):
    model.train()
    y_hats, ys = zip(*[(model(snapshot.x, snapshot.edge_index, snapshot.edge_attr), snapshot.y)
                       for time, snapshot in enumerate(train_dataset)])
    y_hats, ys = torch.stack(list(y_hats)).squeeze(), torch.stack(list(ys))
    loss = mse(y_hats, ys)
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

    if track_with_wandb:
        model.eval()
        with torch.no_grad():
            wandb.log({"epoch": epoch,
                    "train/rmse": rmse(y_hats, ys),
                    "train/mae": mae(y_hats, ys),
                    "train/acc": accuracy(y_hats, ys),
                    "train/f1": f1(y_hats, ys) })
            y_hats, ys = zip(*[(model(snapshot.x, snapshot.edge_index, snapshot.edge_attr), snapshot.y)
                            for time, snapshot in enumerate(val_dataset)])
            y_hats, ys = torch.stack(list(y_hats)).squeeze(), torch.stack(list(ys))
            wandb.log({"epoch": epoch,
                    "val/rmse": rmse(y_hats, ys),
                    "val/mae": mae(y_hats, ys),
                    "val/acc": accuracy(y_hats, ys),
                    "val/f1": f1(y_hats, ys) })


wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: kevinxli. Use `wandb login --relogin` to force relogin


 72%|███████▏  | 36/50 [10:00<03:53, 16.69s/it]


KeyboardInterrupt: 

In [6]:
if track_with_wandb:
    model.eval()
    with torch.no_grad():
        y_hats, ys = zip(*[(model(snapshot.x, snapshot.edge_index, snapshot.edge_attr), snapshot.y)
                            for time, snapshot in enumerate(test_dataset)])
        y_hats, ys = torch.stack(list(y_hats)).squeeze(), torch.stack(list(ys))
        wandb.log({"epoch": epoch,
                "test/rmse": rmse(y_hats, ys),
                "test/mae": mae(y_hats, ys),
                "test/acc": accuracy(y_hats, ys),
                "test/f1": f1(y_hats, ys) })